# STOCK CLOSING PRICE PREDICTION

In [62]:
#importing libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import pandas_datareader as pdr

key="30c0973b8a648106ae38faea9031b9a3924c7469"

In [63]:
#Getting Dataset
df=pdr.get_data_tiingo('DIS', api_key=key)

C:\Users\sneha\AppData\Local\Temp\ipykernel_13724\3742500115.py:2: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.



In [64]:
df.to_csv('DIS.csv')

In [65]:
df=pd.read_csv('DIS.csv', date_parser=True)

In [66]:
df.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,DIS,2018-10-29 00:00:00+00:00,113.04,114.94,111.96,114.45,9234069,110.803854,112.666268,109.745218,112.185961,9234069,0.0,1.0
1,DIS,2018-10-30 00:00:00+00:00,114.76,114.89,112.41,113.39,7365577,112.489829,112.617257,110.186316,111.146930,7365577,0.0,1.0
2,DIS,2018-10-31 00:00:00+00:00,114.83,116.52,114.68,116.23,9196670,112.558444,114.215013,112.411411,113.930749,9196670,0.0,1.0
3,DIS,2018-11-01 00:00:00+00:00,116.10,116.28,114.90,115.20,6779884,113.803321,113.979760,112.627059,112.921125,6779884,0.0,1.0
4,DIS,2018-11-02 00:00:00+00:00,115.18,116.95,114.37,116.83,6377852,112.901520,114.636507,112.107544,114.518880,6377852,0.0,1.0


In [67]:
df.tail()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
1253,DIS,2023-10-23 00:00:00+00:00,83.10,84.0250,80.910,82.08,10772524,83.10,84.0250,80.910,82.08,10772524,0.0,1.0
1254,DIS,2023-10-24 00:00:00+00:00,82.56,84.1997,81.740,83.41,11499469,82.56,84.1997,81.740,83.41,11499469,0.0,1.0
1255,DIS,2023-10-25 00:00:00+00:00,80.59,82.5500,80.515,82.13,9372522,80.59,82.5500,80.515,82.13,9372522,0.0,1.0
1256,DIS,2023-10-26 00:00:00+00:00,79.78,80.6100,79.420,80.08,11650989,79.78,80.6100,79.420,80.08,11650989,0.0,1.0
1257,DIS,2023-10-27 00:00:00+00:00,79.33,81.0250,79.230,79.75,10300556,79.33,81.0250,79.230,79.75,10300556,0.0,1.0


## Data Pre-processing

In [68]:
df['date'] = pd.to_datetime(df['date'])
# df['date'] = df['date'].dt.date
df

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,DIS,2018-10-29 00:00:00+00:00,113.04,114.9400,111.960,114.45,9234069,110.803854,112.666268,109.745218,112.185961,9234069,0.0,1.0
1,DIS,2018-10-30 00:00:00+00:00,114.76,114.8900,112.410,113.39,7365577,112.489829,112.617257,110.186316,111.146930,7365577,0.0,1.0
2,DIS,2018-10-31 00:00:00+00:00,114.83,116.5200,114.680,116.23,9196670,112.558444,114.215013,112.411411,113.930749,9196670,0.0,1.0
3,DIS,2018-11-01 00:00:00+00:00,116.10,116.2800,114.900,115.20,6779884,113.803321,113.979760,112.627059,112.921125,6779884,0.0,1.0
4,DIS,2018-11-02 00:00:00+00:00,115.18,116.9500,114.370,116.83,6377852,112.901520,114.636507,112.107544,114.518880,6377852,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,DIS,2023-10-23 00:00:00+00:00,83.10,84.0250,80.910,82.08,10772524,83.100000,84.025000,80.910000,82.080000,10772524,0.0,1.0
1254,DIS,2023-10-24 00:00:00+00:00,82.56,84.1997,81.740,83.41,11499469,82.560000,84.199700,81.740000,83.410000,11499469,0.0,1.0
1255,DIS,2023-10-25 00:00:00+00:00,80.59,82.5500,80.515,82.13,9372522,80.590000,82.550000,80.515000,82.130000,9372522,0.0,1.0
1256,DIS,2023-10-26 00:00:00+00:00,79.78,80.6100,79.420,80.08,11650989,79.780000,80.610000,79.420000,80.080000,11650989,0.0,1.0


In [69]:
# Splitting train and test set
df_training=df[df['date']<='2022-01-01'].copy()
df_test=df[df['date']>'2022-01-01'].copy()
df_test, df_training

(     symbol                      date   close      high       low    open  \
 800     DIS 2022-01-03 00:00:00+00:00  156.76  157.5600  155.3600  155.83   
 801     DIS 2022-01-04 00:00:00+00:00  155.73  160.3200  155.5500  158.59   
 802     DIS 2022-01-05 00:00:00+00:00  155.19  159.3800  155.1000  156.52   
 803     DIS 2022-01-06 00:00:00+00:00  156.90  157.7700  153.6791  156.24   
 804     DIS 2022-01-07 00:00:00+00:00  157.83  159.3000  156.2900  156.90   
 ...     ...                       ...     ...       ...       ...     ...   
 1253    DIS 2023-10-23 00:00:00+00:00   83.10   84.0250   80.9100   82.08   
 1254    DIS 2023-10-24 00:00:00+00:00   82.56   84.1997   81.7400   83.41   
 1255    DIS 2023-10-25 00:00:00+00:00   80.59   82.5500   80.5150   82.13   
 1256    DIS 2023-10-26 00:00:00+00:00   79.78   80.6100   79.4200   80.08   
 1257    DIS 2023-10-27 00:00:00+00:00   79.33   81.0250   79.2300   79.75   
 
         volume  adjClose   adjHigh    adjLow  adjOpen  adjVol

In [70]:
training_df=df_training.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
training_df

,close,high,low,open,volume
0,113.04,114.94,111.960,114.45,9234069
1,114.76,114.89,112.410,113.39,7365577
2,114.83,116.52,114.680,116.23,9196670
3,116.10,116.28,114.900,115.20,6779884
4,115.18,116.95,114.370,116.83,6377852
...,...,...,...,...,...
795,152.80,154.02,151.400,152.88,7765470
796,155.20,156.50,152.410,152.62,12198679
797,154.87,155.92,154.180,154.56,8737004
798,155.93,157.07,155.515,155.71,7228438


## Feature Scaling

In [71]:
scaler=MinMaxScaler(feature_range=(0,1))
training_df=scaler.fit_transform(training_df)
training_df

array([[0.2348687 , 0.23898393, 0.28273016, 0.25895674, 0.07690548],
       [0.24967714, 0.23855193, 0.28659847, 0.24979472, 0.05484269],
       [0.25027981, 0.25263522, 0.30611192, 0.27434202, 0.07646388],
       ...,
       [0.59500646, 0.5930534 , 0.6456632 , 0.60564415, 0.07103623],
       [0.60413259, 0.60298946, 0.65713917, 0.61558408, 0.05322338],
       [0.59517865, 0.59866943, 0.65047709, 0.61696703, 0.04096006]])

## Converting into Dataset Matrix (taking previous 60 values)

In [72]:
X_train=[]
y_train=[]

In [73]:
training_df.shape

(800, 5)

In [74]:
ltrain=training_df.shape[0]

for i in range(60, ltrain):
    X_train.append(training_df[i-60:i])
    y_train.append(training_df[i,0])

In [75]:
X_train, y_train=np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((740, 60, 5), (740,))

## Building LSTM 

In [76]:
regressior=Sequential()
regressior.add(LSTM(units=60, activation='relu', return_sequences=True, input_shape=(X_train.shape[1],5)))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=60, activation='relu', return_sequences=True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=80, activation='relu', return_sequences=True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=120, activation='relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units=1))

regressior.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 60, 60)            15840     
                                                                 
 dropout_8 (Dropout)         (None, 60, 60)            0         
                                                                 
 lstm_9 (LSTM)               (None, 60, 60)            29040     
                                                                 
 dropout_9 (Dropout)         (None, 60, 60)            0         
                                                                 
 lstm_10 (LSTM)              (None, 60, 80)            45120     
                                                                 
 dropout_10 (Dropout)        (None, 60, 80)            0         
                                                                 
 lstm_11 (LSTM)              (None, 120)              

In [77]:
regressior.compile(optimizer='adam', loss='mean_squared_error')
regressior.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
12/12 [==============================] - 20s 295ms/step - loss: 0.1667
Epoch 2/50
12/12 [==============================] - 4s 309ms/step - loss: 0.0393
Epoch 3/50
12/12 [==============================] - 4s 293ms/step - loss: 0.0195
Epoch 4/50
12/12 [==============================] - 3s 290ms/step - loss: 0.0125
Epoch 5/50
12/12 [==============================] - 4s 300ms/step - loss: 0.0106
Epoch 6/50
12/12 [==============================] - 4s 326ms/step - loss: 0.0093
Epoch 7/50
12/12 [==============================] - 3s 288ms/step - loss: 0.0101
Epoch 8/50
12/12 [==============================] - 4s 359ms/step - loss: 0.0084
Epoch 9/50
12/12 [==============================] - 4s 317ms/step - loss: 0.0085
Epoch 10/50
12/12 [==============================] - 4s 382ms/step - loss: 0.0078
Epoch 11/50
12/12 [==============================] - 5s 389ms/step - loss: 0.0081
Epoch 12/50
12/12 [==============================] - 5s 378ms/step - loss: 0.0068
Epoch 13/50
12/12 [=====

## Preparing Test Dataset

In [78]:
df_test.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
800,DIS,2022-01-03 00:00:00+00:00,156.76,157.56,155.3600,155.83,10228257,156.76,157.56,155.3600,155.83,10228257,0.0,1.0
801,DIS,2022-01-04 00:00:00+00:00,155.73,160.32,155.5500,158.59,16582041,155.73,160.32,155.5500,158.59,16582041,0.0,1.0
802,DIS,2022-01-05 00:00:00+00:00,155.19,159.38,155.1000,156.52,12272097,155.19,159.38,155.1000,156.52,12272097,0.0,1.0
803,DIS,2022-01-06 00:00:00+00:00,156.90,157.77,153.6791,156.24,11095326,156.90,157.77,153.6791,156.24,11095326,0.0,1.0
804,DIS,2022-01-07 00:00:00+00:00,157.83,159.30,156.2900,156.90,9554622,157.83,159.30,156.2900,156.90,9554622,0.0,1.0


In [79]:
df_training.tail()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
795,DIS,2021-12-27 00:00:00+00:00,152.80,154.02,151.400,152.88,7765470,152.80,154.02,151.400,152.88,7765470,0.0,1.0
796,DIS,2021-12-28 00:00:00+00:00,155.20,156.50,152.410,152.62,12198679,155.20,156.50,152.410,152.62,12198679,0.0,1.0
797,DIS,2021-12-29 00:00:00+00:00,154.87,155.92,154.180,154.56,8737004,154.87,155.92,154.180,154.56,8737004,0.0,1.0
798,DIS,2021-12-30 00:00:00+00:00,155.93,157.07,155.515,155.71,7228438,155.93,157.07,155.515,155.71,7228438,0.0,1.0
799,DIS,2021-12-31 00:00:00+00:00,154.89,156.57,154.740,155.87,6189861,154.89,156.57,154.740,155.87,6189861,0.0,1.0


In [80]:
past_60_days=df_training.tail(60)
# past_60_days
df1=past_60_days.append(df_test) #past 60 days taken for the first value of test data
df1=df1.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
df1

C:\Users\sneha\AppData\Local\Temp\ipykernel_13724\1121012054.py:3: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,close,high,low,open,volume
740,177.71,179.6300,177.3300,177.70,7779997
741,176.74,178.6000,176.4029,177.93,4132182
742,173.52,176.5200,173.4800,176.02,8046270
743,173.13,174.4900,172.5450,173.62,5188380
744,172.96,173.7000,170.9400,173.63,8918152
...,...,...,...,...,...
1253,83.10,84.0250,80.9100,82.08,10772524
1254,82.56,84.1997,81.7400,83.41,11499469
1255,80.59,82.5500,80.5150,82.13,9372522
1256,79.78,80.6100,79.4200,80.08,11650989


In [81]:
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(df1)
df1

# inputs=scaler.transform(df1)
# inputs

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 9.90059243e-01,
        5.51518266e-02],
       [9.90141275e-01, 9.89683080e-01, 9.90597277e-01, 9.92368712e-01,
        0.00000000e+00],
       [9.57414371e-01, 9.68848912e-01, 9.60952990e-01, 9.73190079e-01,
        5.91776455e-02],
       ...,
       [1.29078158e-02, 2.76052726e-02, 1.80944856e-02, 3.04247414e-02,
        7.92294355e-02],
       [4.67527188e-03, 8.17340438e-03, 6.98890761e-03, 9.84034542e-03,
        1.13677898e-01],
       [1.01636345e-04, 1.23302216e-02, 5.06191233e-03, 6.52675971e-03,
        9.32605117e-02]])

In [82]:
df1.shape

(518, 5)

In [83]:
X_test=[]
y_test=[]

In [84]:
ltest=df1.shape[0]
for i in range(60,ltest):
    X_test.append(df1[i-60:i])
    y_test.append(df1[i,0])

In [85]:
X_test, y_test=np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((458, 60, 5), (458,))

## Predicting the stock closing price

In [86]:
y_pred=regressior.predict(X_test)

15/15 [==============================] - 4s 155ms/step


In [87]:
# scaler.scale_

In [88]:
# scale=1/1.
# scale
# y_pred=y_pred*scale
# y_test=y_test*scale

In [89]:
y_test

array([7.87071857e-01, 7.76603313e-01, 7.71114951e-01, 7.88494766e-01,
       7.97946946e-01, 7.85445675e-01, 7.98556764e-01, 7.97642037e-01,
       7.73655859e-01, 7.38083139e-01, 7.41437138e-01, 7.19483687e-01,
       6.94176237e-01, 5.90100620e-01, 5.90913711e-01, 5.81258258e-01,
       5.51682082e-01, 5.70179896e-01, 6.02805163e-01, 6.46915337e-01,
       6.62364061e-01, 6.43358065e-01, 6.17034251e-01, 6.37259884e-01,
       6.42240065e-01, 6.41935156e-01, 6.90212420e-01, 7.40319138e-01,
       7.12978961e-01, 7.27004777e-01, 7.66338042e-01, 7.82904767e-01,
       7.48348409e-01, 7.32188231e-01, 6.98953146e-01, 6.76389877e-01,
       7.12267507e-01, 7.13588779e-01, 7.02713690e-01, 6.74662059e-01,
       6.91330420e-01, 6.73340787e-01, 6.24047159e-01, 5.50665718e-01,
       5.32879358e-01, 5.52190263e-01, 5.52088627e-01, 5.32879358e-01,
       5.05234272e-01, 5.57780262e-01, 5.97824982e-01, 6.11342616e-01,
       6.19778433e-01, 6.02703527e-01, 6.17847342e-01, 5.92743165e-01,
      

## Visualization the results

In [113]:
# import matplotlib.pyplot as plt

# # Assuming you have the 'y_test' and 'y_pred' values as NumPy arrays

# # Create an array of x-axis values for the data points
# x_values = range(len(y_test))

# # Create the plot
# plt.figure(figsize=(14, 5))

# # Plot 'y_test' with red markers
# plt.plot(x_values, y_test, color='red', marker='o', linestyle='-', label='Real Stock Price')

# # Plot 'y_pred' with blue markers
# plt.plot(x_values, y_pred, color='blue', marker='o',  linestyle='-', label='Predicted Stock Price')

# # Add labels and legend
# plt.title('Stock Price Prediction')
# plt.xlabel('Time')
# plt.ylabel('Stock Price')
# plt.legend()

# # Show the plot
# plt.show()


In [112]:
import plotly.graph_objs as go

# Create a trace for actual values
trace_actual = go.Scatter(x=df_test['date'], y=y_test, mode='lines', name='Actual', line=dict(color='red'))

# Create a trace for predicted values
trace_pred = go.Scatter(x=df_test['date'], y=y_pred.flatten(), mode='lines', name='Predicted', line=dict(color='green'))

# Create a trace for markers with reduced size
trace_markers1 = go.Scatter(
    x=df_test['date'],
    y=y_test.flatten(),
    mode='markers',
    name='',
    showlegend=False,
    marker=dict(color='red', size=3),  # Adjust the size here
)

trace_markers2 = go.Scatter(
    x=df_test['date'],
    y=y_pred.flatten(),
    mode='markers',
    name='',
    showlegend=False,
    marker=dict(color='green', size=3),  # Adjust the size here
)

# Create the Plotly figure
fig = go.Figure(data=[trace_actual, trace_pred, trace_markers1, trace_markers2])

# Customize the layout
fig.update_layout(
    title='Stock Price Prediction',
    xaxis_title='Time',
    yaxis_title='Stock Price',
     width=800,
    height=600
)

# Show the Plotly figure
fig.show()
